<a href="https://colab.research.google.com/github/Ahmad-abdulrahim/Personalized-Job-Matching-System/blob/Development/PJMS_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical computations
import re           # For regular expressions
import nltk         # Natural Language Toolkit
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
from nltk.corpus import stopwords   # Stopwords
from nltk.tokenize import word_tokenize  # Tokenization
from nltk.stem import WordNetLemmatizer  # Lemmatization
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
df = pd.read_csv("/content/Job Postings Dataset.csv")
df.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


In [ ]:
df.columns

Index(['jobpost', 'date', 'Title', 'Company', 'AnnouncementCode', 'Term',
       'Eligibility', 'Audience', 'StartDate', 'Duration', 'Location',
       'JobDescription', 'JobRequirment', 'RequiredQual', 'Salary',
       'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 'Attach',
       'Year', 'Month', 'IT'],
      dtype='object')

In [ ]:
df.shape

(19001, 24)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19001 entries, 0 to 19000
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   jobpost           19001 non-null  object
 1   date              19001 non-null  object
 2   Title             18973 non-null  object
 3   Company           18994 non-null  object
 4   AnnouncementCode  1208 non-null   object
 5   Term              7676 non-null   object
 6   Eligibility       4930 non-null   object
 7   Audience          640 non-null    object
 8   StartDate         9675 non-null   object
 9   Duration          10798 non-null  object
 10  Location          18969 non-null  object
 11  JobDescription    15109 non-null  object
 12  JobRequirment     16479 non-null  object
 13  RequiredQual      18517 non-null  object
 14  Salary            9622 non-null   object
 15  ApplicationP      18941 non-null  object
 16  OpeningDate       18295 non-null  object
 17  Deadline    

In [ ]:
# Dropping unnecessary columns from dataframe
df_clean = df.drop(columns=['date', 'AnnouncementCode',
                            'StartDate', 'ApplicationP',
                            'OpeningDate', 'Deadline',
                            'Notes', 'Attach', 'Year',
                            'Month', 'AboutC', 'jobpost',
                            'Term', 'Eligibility', 'Audience',
                            'Duration', 'Location', 'Salary'])

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19001 entries, 0 to 19000
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Title           18973 non-null  object
 1   Company         18994 non-null  object
 2   JobDescription  15109 non-null  object
 3   JobRequirment   16479 non-null  object
 4   RequiredQual    18517 non-null  object
 5   IT              19001 non-null  bool  
dtypes: bool(1), object(5)
memory usage: 760.9+ KB


In [ ]:
df_clean.columns = df_clean.columns.str.strip()
df_clean.rename(columns={'RequiredQual': 'RequiredQualifications', 'JobRequirment': 'JobRequirement'}, inplace=True)

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19001 entries, 0 to 19000
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Title                   18973 non-null  object
 1   Company                 18994 non-null  object
 2   JobDescription          15109 non-null  object
 3   JobRequirement          16479 non-null  object
 4   RequiredQualifications  18517 non-null  object
 5   IT                      19001 non-null  bool  
dtypes: bool(1), object(5)
memory usage: 760.9+ KB


In [ ]:
# List of columns to consider for dropping rows
columns_to_check = ['Title', 'Company', 'JobDescription', 'JobRequirement', 'RequiredQualifications']

# Drop rows with null values in the specified columns
df_clean.dropna(subset=columns_to_check, inplace=True)

# Reset index after dropping rows
df_clean.reset_index(drop=True, inplace=True)

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13124 entries, 0 to 13123
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Title                   13124 non-null  object
 1   Company                 13124 non-null  object
 2   JobDescription          13124 non-null  object
 3   JobRequirement          13124 non-null  object
 4   RequiredQualifications  13124 non-null  object
 5   IT                      13124 non-null  bool  
dtypes: bool(1), object(5)
memory usage: 525.6+ KB


In [ ]:
df_clean.head()

,Title,Company,JobDescription,JobRequirement,RequiredQualifications,IT
0,Chief Financial Officer,AMERIA Investment Consulting Company,AMERIA Investment Consulting Company is seekin...,- Supervises financial management and administ...,"To perform this job successfully, an\r\nindivi...",False
1,Country Coordinator,Caucasus Environmental NGO Network (CENN),Public outreach and strengthening of a growing...,- Working with the Country Director to provide...,"- Degree in environmentally related field, or ...",False
2,BCC Specialist,Manoff Group,The LEAD (Local Enhancement and Development fo...,- Identify gaps in knowledge and overseeing in...,"- Advanced degree in public health, social sci...",False
3,"Community Development, Capacity Building and C...",Food Security Regional Cooperation and Stabili...,Food Security Regional Cooperation and Stabili...,- Assist the Tavush Marz communities and commu...,- Higher Education and/or professional experie...,False
4,Country Economist (NOB),"United Nations Development Programme, Armenia",The United Nations Development Programme in Ar...,The incumbent under direct supervision of UNDP...,- Minimum Masters Degree in Economics;\r\n- Mi...,False


In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])  # Disable parser and NER for speed

In [ ]:
def preprocess_text_spacy(text):
    """
    Advanced text preprocessing using spaCy.
    """
    # Initial cleaning (remove punctuation and numbers, optional)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()

    # Process text with spaCy
    doc = nlp(text)

    # Lemmatize and remove stopwords and punctuations
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Join processed tokens back into string
    return " ".join(tokens)

def preprocess_dataframe(df, text_columns):
    """
    Apply text preprocessing to specified text columns of a pandas DataFrame.
    """
    # Use .applymap() for elementwise operation if necessary
    for col in text_columns:
        df[col] = df[col].apply(preprocess_text_spacy)
    return df

In [ ]:
text_columns = ['JobRequirement', 'JobDescription', 'RequiredQualifications', 'Title', 'Company']
df_preprocessed = preprocess_dataframe(df_clean.copy(), text_columns)

In [ ]:
df_preprocessed.head()

,Title,Company,JobDescription,JobRequirement,RequiredQualifications,IT
0,chief financial officer,ameria investment consult company,ameria investment consult company seek chief f...,supervise financial management administrative ...,perform job successfully individual able perfo...,False
1,country coordinator,caucasus environmental ngo network cenn,public outreach strengthen grow network enviro...,work country director provide environmental in...,degree environmentally related field year rele...,False
2,bcc specialist,manoff group,lead local enhancement development health bcc ...,identify gap knowledge oversee information col...,advanced degree public health social science c...,False
3,community development capacity building conflict,food security regional cooperation stability p...,food security regional cooperation stability p...,assist tavush marz community community union d...,high education andor professional experience e...,False
4,country economist nob,united nations development programme armenia,united nations development programme armenia a...,incumbent direct supervision undp resident rep...,minimum master degree economic minimum year re...,False


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Concatenate text columns
df_preprocessed['combined_text'] = df_preprocessed['JobDescription'] + ' ' + df_preprocessed['JobRequirement'] + ' ' + df_preprocessed['RequiredQualifications']

# Apply TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000, ngram_range=(1, 2))
tfidf_matrix = vectorizer.fit_transform(df_preprocessed['combined_text'])

In [ ]:
# Inspect top N terms by TF-IDF score
feature_names = vectorizer.get_feature_names_out()
top_n = 20
sorted_features = np.argsort(tfidf_matrix.toarray().sum(axis=0))[-top_n:]
print(np.array(feature_names)[sorted_features])

['service' 'sale' 'responsible' 'develop' 'management' 'excellent'
 'company' 'customer' 'software' 'design' 'team' 'language' 'good'
 'development' 'ability' 'project' 'experience' 'skill' 'work' 'knowledge']


For a job matching system where understanding the nuanced meaning of job descriptions and resumes is crucial, Word Embeddings or Doc2Vec could provide significant benefits over TF-IDF by capturing semantic similarities.

In [ ]:
df_preprocessed[['RequiredQualifications']].head()

,RequiredQualifications
0,perform job successfully individual able perfo...
1,degree environmentally related field year rele...
2,advanced degree public health social science c...
3,high education andor professional experience e...
4,minimum master degree economic minimum year re...


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2024-02-13 16:05:56--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-02-13 16:05:57--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-02-13 16:05:57--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors

# Convert GloVe to Word2Vec format
glove_input_file = 'glove.6B.200d.txt'
word2vec_output_file = 'glove.6B.200d.word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

# Load the converted model
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

<ipython-input-22-d28c250c4df9>:7: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


In [ ]:
similar_words = model.most_similar('computer', topn=10)
print(similar_words)

[('computers', 0.8357723951339722), ('software', 0.7828460335731506), ('technology', 0.6907660961151123), ('pc', 0.6647984385490417), ('systems', 0.658431887626648), ('internet', 0.6446206569671631), ('electronic', 0.6443334817886353), ('hardware', 0.6368629932403564), ('ibm', 0.6295379996299744), ('computing', 0.6258440017700195)]


In [ ]:
# Vocabulary Overlap Check

glove_vocab = set(model.key_to_index.keys())

# Extract unique words from dataset
job_posts_words = set(" ".join(df_preprocessed['combined_text']).split())
missing_words = job_posts_words - glove_vocab

# Percentage of words in dataset covered by GloVe model
coverage = len(job_posts_words - missing_words) / len(job_posts_words) * 100
print(f"Coverage: {coverage:.2f}%")
print(f"Number of missing words: {len(missing_words)}")

Coverage: 62.30%
Number of missing words: 7447


A coverage of 62.30% with 7,447 missing words from our job posts dataset in the GloVe vocabulary highlights a significant gap in the embeddings' ability to fully represent the specific terms used in our domain. This situation is not uncommon, especially for
specialized or technical domains where jargon, brand names, acronyms, and newer terms may not be included in general-purpose embeddings trained on broader corpora.

The following strategy can be implemented to address this gap:

1. Train Additional Embeddings: since we have substantial corpus of domain-specific text in our dataset, we can train our word embeddings on this corpus. Tools like FastText can learn the nuances of the specific domain.

2. Combine Embeddings: Using our newly trained embeddings in conjunction with GloVe. For missing words, we can fallback on our domain-specific embeddings.

In [ ]:
!pip install gensim fasttext
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English model, ~300MB
ft_model = fasttext.load_model('cc.en.300.bin')

In [ ]:
def get_combined_embedding(word, glove_model, fasttext_model):
    """
    Attempt to fetch the word's embedding from GloVe. If not found, get it from FastText.
    """
    # Try to get from GloVe
    try:
        return glove_model[word]
    except KeyError:
        # If not found in GloVe, try FastText
        return fasttext_model.get_word_vector(word)

In [ ]:
# Example usage (assuming `model` is your GloVe model loaded as shown earlier)
word = 'python'
combined_vector = get_combined_embedding(word, model, ft_model)
print(f"Combined vector for {word}: {combined_vector[:10]}")  # Print first 10 elements for brevity